In [ ]:
!pip install --quiet ipywidgets accelerate bitsandbytes huggingface_hub transformers 

In [5]:
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_VARIANT = "27b-chat"  # @param ["2b-predict", "9b-chat", "9b-predict", "27b-chat", "27b-predict"]

model_id = f"google/txgemma-{MODEL_VARIANT}"

if MODEL_VARIANT == "2b-predict":
    additional_args = {}
else:
    additional_args = {
        "quantization_config": BitsAndBytesConfig(load_in_8bit=True)
    }

tokenizer = AutoTokenizer.from_pretrained(model_id)
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True,  # ✅ allow offloading to CPU
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto", 
    quantization_config=bnb_config,
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Device set to use cuda:0


In [4]:
drug_list = ["Dabrafenib", "Methotrexate", "Delamanid", "Demeclocycline", "5-Fluorouracil"]
drug = drug_list[0]

# Stage 1: Get drug information
prompt = f"""
Instructions: Answer the following question about drug properties.
Context: {drug} is a small molecule drug
Question: Find me the drug information about this {drug}
Answer:"""
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids, max_new_tokens=100)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

# Extract drug information from the first response
drug_info = response.split("Answer:")[-1].strip()

# Stage 2: Get pathway involvement
prompt = f"""
Instructions: Answer the following question about drug involvement in pathway.
Context: {drug_info}
Question: Show me the most relavant pathways involvement for {drug}
Answer:"""
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids, max_new_tokens=100)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

# Stage 3: Predict drug effect given gene upregulation
genes_list = ['ENSG00000283959', 'KLKP1-1', 'BBOX1-AS1', 'ENSG00000257732', 'ENSG00000285708', 'ENSG00000287682', 'MICOS13', 'ENSG00000286076']
prompt = f"""
Instructions: Answer the following question about drug effects.
Context: {drug_info}
Question: The genes {genes_list} are all upregulated, predict the effect of {drug}
Answer:"""
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids, max_new_tokens=100)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)



Instructions: Answer the following question about drug properties.
Context: Dabrafenib is a small molecule drug
Question: Find me the drug information about this Dabrafenib
Answer: Dabrafenib is a potent and selective BRAF kinase inhibitor. It is used in the treatment of melanoma and other cancers with BRAF gene mutations. 


Instructions: Answer the following question about drug involvement in pathway.
Context: Dabrafenib is a potent and selective BRAF kinase inhibitor. It is used in the treatment of melanoma and other cancers with BRAF gene mutations.
Question: Show me the most relavant pathways involvement for Dabrafenib
Answer:
BRAF Signaling Pathway
PI3K/AKT Signaling Pathway
MAPK Signaling Pathway

Instructions: Answer the following question about drug effects.
Context: Dabrafenib is a potent and selective BRAF kinase inhibitor. It is used in the treatment of melanoma and other cancers with BRAF gene mutations.
Question: The genes ['ENSG00000283959', 'KLKP1-1', 'BBOX1-AS1', 'ENS

In [7]:
drug = "Ponatinib"

# Stage 1: Get drug information
prompt = f"""
Instructions: Answer the following question about drug properties.
Context: {drug} is a small molecule drug
Question: Find me the drug information about this {drug}
Answer:"""
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids, max_new_tokens=100)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

# Extract drug information from the first response
drug_info = response.split("Answer:")[-1].strip()

# Stage 2: Get pathway involvement
prompt = f"""
Instructions: Answer the following question about drug involvement in pathway.
Context: {drug_info}
Question: Show me the most relavant pathways involvement for {drug}
Answer:"""
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids, max_new_tokens=100)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

# Stage 3: Predict drug effect given gene upregulation
# genes_list = ['ENSG00000283959', 'KLKP1-1', 'BBOX1-AS1', 'ENSG00000257732', 'ENSG00000285708', 'ENSG00000287682', 'MICOS13', 'ENSG00000286076']
prompt = f"""
Instructions: Answer the following question about drug effects.
Context: {drug_info}
Question: The transmembrane transporter binding activity is downregulated, and Arachidonic acid metabolism is upregulated, is it the same effect as the {drug} MOA?
Answer:"""
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids, max_new_tokens=100)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)



Instructions: Answer the following question about drug properties.
Context: Ponatinib is a small molecule drug
Question: Find me the drug information about this Ponatinib
Answer: Ponatinib is a potent inhibitor of the tyrosine kinases Bcr-Abl, Lyn, and Flt3. It is used to treat chronic myeloid leukemia (CML) and acute lymphoblastic leukemia (ALL).

Instructions: Answer the following question about drug involvement in pathway.
Context: Ponatinib is a potent inhibitor of the tyrosine kinases Bcr-Abl, Lyn, and Flt3. It is used to treat chronic myeloid leukemia (CML) and acute lymphoblastic leukemia (ALL).
Question: Show me the most relavant pathways involvement for Ponatinib
Answer:
* **Tyrosine Kinase Signaling Pathway:** Ponatinib directly inhibits three tyrosine kinases: Bcr-Abl, Lyn, and Flt3. These kinases play crucial roles in various cellular processes, including cell growth, proliferation, differentiation, and survival. 
* **Leukemia Development:** Ponatinib's therapeutic effect 

In [8]:
import numpy as np
import torch
import torch.nn.functional as F
from scipy.stats import pearsonr
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

def compute_confidence_loss_correlation(model, tokenizer, text):
    model.eval()
    device = model.device
    inputs = tokenizer(text, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        logits = outputs.logits

    shifted_logits = logits[:, :-1, :]
    shifted_labels = inputs["input_ids"][:, 1:]
    probs = F.softmax(shifted_logits, dim=-1)

    confidences, _ = probs.max(dim=-1)
    true_token_probs = probs.gather(2, shifted_labels.unsqueeze(-1)).squeeze(-1)
    loss_per_token = -torch.log(true_token_probs + 1e-12)

    confidences_np = confidences.flatten().cpu().numpy()
    loss_np = loss_per_token.flatten().cpu().numpy()

    # 🔧 Filter invalid values
    valid_mask = np.isfinite(loss_np)
    confidences_np = confidences_np[valid_mask]
    loss_np = loss_np[valid_mask]

    print(f"#valid tokens: {len(confidences_np)}")
    print(f"Confidences mean/std: {np.mean(confidences_np):.4f} / {np.std(confidences_np):.4f}")
    print(f"Loss mean/std: {np.mean(loss_np):.4f} / {np.std(loss_np):.4f}")
    print(f"Confidences min/max: {confidences_np.min():.4f} / {confidences_np.max():.4f}")
    print(f"Loss min/max: {loss_np.min():.4f} / {loss_np.max():.4f}")

    if len(confidences_np) < 2 or np.std(confidences_np) == 0 or np.std(loss_np) == 0:
        print("⚠️ Not enough valid data for correlation.")
        return float("nan"), confidences_np, loss_np

    corr, _ = pearsonr(confidences_np, loss_np)
    return corr, confidences_np, loss_np


# 🔬 Input prompt (longer for better token count)
text = (
    "The genes ['ENSG00000283959', 'KLKP1-1', 'BBOX1-AS1'] are upregulated in this sample. "
    "We are interested in evaluating the potential therapeutic impact of Dabrafenib on these genes. "
    "Provide a detailed mechanistic rationale for how Dabrafenib might affect downstream signaling pathways, "
    "especially focusing on any known or predicted interactions with MAPK signaling."
)

# ✅ Assume google/txgemma-2b-chat is already loaded
print("\n🟢 Computing for pre-loaded model: google/txgemma-2b-chat")
corr, confidences_np, loss_np = compute_confidence_loss_correlation(model, tokenizer, text)

if not np.isnan(corr):
    print(f"📉 (confidence vs. token-level loss): {corr:.3f}")
else:
    print("⚠️ Correlation computation failed for google/txgemma-2b-chat")

# 📦 List of other models to compare
model_ids = [
    "openai-community/gpt2",
    "mistralai/Mistral-7B-Instruct-v0.3"
]

# 🚀 Compare each model
for model_id in model_ids:
    try:
        print(f"\n🔍 Loading model: {model_id}")
        
        bnb_config = BitsAndBytesConfig(
            load_in_8bit=True,
            llm_int8_enable_fp32_cpu_offload=True
        )
        
        model_tmp = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            quantization_config=bnb_config,
        )
        tokenizer_tmp = AutoTokenizer.from_pretrained(model_id)

        corr, confidences_np, loss_np = compute_confidence_loss_correlation(model_tmp, tokenizer_tmp, text)
        
        if np.isnan(corr):
            continue 
        
        print(f"📉 (confidence vs. token-level loss): {corr:.3f}")
    
    except Exception as e:
        print(f"❌ Failed for model {model_id}: {e}")



🟢 Computing for pre-loaded model: google/txgemma-2b-chat
#valid tokens: 81
Confidences mean/std: 0.8306 / 0.2305
Loss mean/std: 4.2227 / 5.7773
Confidences min/max: 0.2489 / 1.0000
Loss min/max: -0.0000 / 16.6406
📉 (confidence vs. token-level loss): -0.239

🔍 Loading model: openai-community/gpt2


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


#valid tokens: 84
Confidences mean/std: 0.3237 / 0.2864
Loss mean/std: 4.0703 / 3.2754
Confidences min/max: 0.0128 / 0.9961
Loss min/max: 0.0039 / 14.1484
📉 (confidence vs. token-level loss): -0.542

🔍 Loading model: mistralai/Mistral-7B-Instruct-v0.3


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

#valid tokens: 103
Confidences mean/std: 0.6030 / 0.3210
Loss mean/std: 2.4609 / 3.2676
Confidences min/max: 0.0200 / 1.0000
Loss min/max: -0.0000 / 15.9453
📉 (confidence vs. token-level loss): -0.541


In [8]:
drug = "DTP3" 

# Stage 1: Get drug information
prompt = f"""
Instructions: Answer the following question about drug properties.
Context: {drug} is a small molecule drug
Question: Find me the drug information about this {drug}
Answer:"""
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids, max_new_tokens=100)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

# Extract drug information from the first response
drug_info = response.split("Answer:")[-1].strip()

# Stage 2: Get pathway involvement
prompt = f"""
Instructions: Answer the following question about drug involvement in pathway.
Context: {drug_info}
Question: What does {drug} do? What are the upregulated and downregulatedpathways it affects.
Answer:"""
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids, max_new_tokens=100)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


Instructions: Answer the following question about drug properties.
Context: DTP3 is a small molecule drug
Question: Find me the drug information about this DTP3
Answer: DTP3 is an experimental drug. There is no publicly available information about its properties, mechanisms of action, or clinical trials. 


Instructions: Answer the following question about drug involvement in pathway.
Context: DTP3 is an experimental drug. There is no publicly available information about its properties, mechanisms of action, or clinical trials.
Question: What does DTP3 do? What are the upregulated and downregulatedpathways it affects.
Answer: This question cannot be answered. There is no publicly available information about DTP3, its mechanisms of action, or the pathways it affects.


In [9]:
model_name = "google/txgemma-27b-chat"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

chat = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Example prompt
prompt = """
MIA PaCa-2 cells treated with DTP3 show the following pathway changes:
- Upregulated: apoptosis, p53 signaling, oxidative stress response
- Downregulated: NF-κB signaling, cell cycle, DNA replication

DTP3 is known to inhibit NF-κB. Do these pathway changes support the expected mechanism of action? Explain.
"""

response = chat(prompt, max_new_tokens=300)[0]["generated_text"]
print(response)

/usr/local/lib/python3.12/dist-packages/accelerate/utils/modeling.py:1569: UserWarning: Current model requires 256 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

Device set to use cpu



A549 cells treated with DTP3 show the following pathway changes:
- Upregulated: apoptosis, p53 signaling, oxidative stress response
- Downregulated: NF-κB signaling, cell cycle, DNA replication

DTP3 is known to inhibit NF-κB. Do these pathway changes support the expected mechanism of action? Explain.
**

**Answer:**

Yes, the pathway changes observed in A549 cells treated with DTP3 support its known mechanism of action as an NF-κB inhibitor. Here's why:

* **NF-κB inhibition:** DTP3 directly inhibits NF-κB, a transcription factor crucial for inflammation, cell survival, and proliferation. The observed downregulation of NF-κB signaling confirms this direct inhibitory effect. 

* **Apoptosis induction:** NF-κB often suppresses apoptosis. By inhibiting NF-κB, DTP3 removes this suppression, leading to the upregulation of apoptosis pathways. 

* **p53 signaling activation:**  p53 is a tumor suppressor protein that can induce apoptosis and cell cycle arrest. NF-κB can negatively regulate p